In [16]:
import json
import numpy as np
import pandas as pd

In [56]:
data_dir = './proximity_2019-06-01/'

file_list = []

for i in range(1, 18):
    if i < 10:
        filename = data_dir + 'CTSIserver{:02d}_proximity_2019-06-01.txt'.format(i)
    else:
        filename = data_dir + 'CTSIserver{}_proximity_2019-06-01.txt'.format(i)

        
    
    with open(filename, 'r') as f:
        tmp_list = []
        for line in f:
            record = line.strip()
            record_json = json.loads(record)
            data_json = record_json['data']

            # Each record: timestamp, center member id, scanned member id, count, rssi, beacon
            for m in data_json['rssi_distances'].keys():
                tmp = np.zeros(6)
                # 1st col: timestamp
                tmp[0] = data_json['timestamp']
                # 2nd col: center member id
                tmp[1] = data_json['member_id']
                # 3rd col: scanned member id
                tmp[2] = int(m)
                # 4th col: count
                tmp[3] = data_json['rssi_distances'][m]['count']
                # 5th col: rssi
                tmp[4] = data_json['rssi_distances'][m]['rssi']
                
                # 6th col: beacon or not
                if int(m) > 10000:
                    tmp[5] = 1
                else:
                    tmp[5] = 0
                        
                tmp_list.append(tmp)
    pd_list = pd.DataFrame(tmp_list, columns=['timestamp', 'center_member_id', 'scanned_member_id', 'count', 'rssi', 'beacon'], dtype=np.int32)        
    file_list.append(pd_list)
                

In [60]:
for i in range(len(file_list)):
    file_list[i].to_csv('./parsed_data/beacon{}'.format(i+1))

In [23]:
member_metadata = pd.read_csv('Member-2019-05-28.csv', dtype=object)
id2key = member_metadata[['key','id']]